# Notebook to loop over events generated in Pythia's c++ version


In [1]:
import numpy as np
import sys
import dimod
import matplotlib.pyplot as plt
import os
import re
import neal

from dwave.system import EmbeddingComposite, DWaveSampler, ReverseAdvanceComposite
from dwave_qbsolv import QBSolv
from pyqubo import Spin, Array
from embedding_generator import generate_embedding

In [2]:
# Define variable here

# Observed big difference in obtained minimum when increasing the number of reads from 100 to 1000
num_reads = 10000
num_events = 1000
path = '/Users/1zd/quantumjets/pythiaEvents/'

theta = np.pi/2.

sampler = EmbeddingComposite(DWaveSampler(solver='Advantage_system1.1'))
sa_sampler = neal.SimulatedAnnealingSampler()

In [10]:

with open('/Users/1zd/quantumjets/pythiaEvents/DWaveResults_penalty_3.dat', 'w') as fout:
    for ev in range(939, num_events):
        print(ev)

        fname = path + 'Event_' + str(ev) + '.dat'
        f = open(fname)

        pat = re.compile(r"\s+")

        g = list(zip(*[ pat.sub(" ",x.strip()).split() for x in f]))

        px = np.array([ float(x) for x in g[0]])
        py = np.array([ float(x) for x in g[1]])
        pz = np.array([ float(x) for x in g[2]])
        e = np.array([ float(x) for x in g[3]])

        # Generate qubo

        n_part = len(px)
        #print(n_part)
        s = Array.create('s', shape=n_part, vartype='BINARY')

        # Create an array of zeroes for qubo matrix coefficients
        coeff = [[0] * n_part for _ in range(n_part)] 

        for i in range(0,n_part):
            for j in range(0,n_part):
                coeff[i][j] = (px[i]*px[j] +py[i]*py[j] + pz[i]*pz[j] - e[i]*e[j] * np.cos(theta))/(1 - np.cos(theta))

        # Construct Hamiltonian
        #H = sum([ ((1.0/(np.sqrt(px[i]*px[i] +py[i]*py[i] + pz[i]*pz[i])*np.sqrt(px[j]*px[j] +py[j]*py[j] + pz[j]*pz[j]))) - 1.0)*coeff[i][j]*s[i]*s[j] for i in range(0,n_part) for j in range(0,n_part)])
        H = sum([ -1.0*coeff[i][j]*s[i]*s[j] for i in range(0,n_part) for j in range(0,n_part)])

        # Compile model using pyqubo

        model = H.compile()
        qubo, offset = model.to_qubo()

        # Submit problem to D-Wave
        #response = QBSolv().sample_qubo(qubo)#
        ch_str = qubo[max(qubo, key=qubo.get)]*0.1
        response = sampler.sample_qubo(qubo,num_reads=5000, chain_strength=ch_str, annealing_time=100)
        #sampler_reverse = ReverseAdvanceComposite(sampler)
        #schedule = [[[0.0, 1.0], [t, 0.5], [20, 1.0]] for t in (5, 10, 15)]
        #sampleset = sampler_reverse.sample_qubo(qubo, anneal_schedules=schedule, num_reads=100,reinitialize_state=True) 


        fnameout = '/Users/1zd/quantumjets/pythiaEvents/Event_%s_qsolutions_v2.dat' %ev
        with open(fnameout, 'w') as f:
            for p in range(n_part):
                key = 's[%s]' %p
                f.write("%s\n" %(response.first.sample[key]))
    
    #print('Finished annealing run on Advantage 1.1')

        #print(response.first.energy)

        #response_sa = sa_sampler.sample_qubo(qubo)

    #print('Finished simmulated annealing routine')

    #print(response_sa.first.energy)

        sum_mom = sum([np.sqrt(px[i]*px[i] + py[i]*py[i]+ pz[i]*pz[i])  for i in range(0,n_part)])
    
        #print(sum_mom)
        thrust = 2.*np.sqrt(-1.*response.first.energy)/sum_mom
        print(thrust)
        fout.write("%s \n" %(thrust))

        #rev = 2.*np.sqrt(-1.*sampleset.first.energy)/sum_mom
        #print(rev)

        #thrust_sa = 2.*np.sqrt(-1.*response_sa.first.energy)/sum_mom
        #fout.write("%s  %s  %s \n" %(thrust,thrust_sa,rev))
    
    #print(response.data_vectors['sample'][0])
    # Store solutions

    #qpu_solutions = []

    #for datum in response.data(['sample','energy']):
    #    print(datum.energy)
    #    qpu_solutions.append(datum.sample)


939
0.9685575037793271
940
0.9719162814953416
941
0.9699583643541713
942
0.971542331163138
943
0.9492612155731849
944
0.9620912667750917
945
0.9317576133478563
946
0.8690522253415082
947
0.9445618705574651
948
0.9412393375245816
949
0.9802923884698803
950
0.9132366617536885
951
0.9579809018757005
952
0.9687592450323753
953
0.9797402647028792
954
0.7401055013961803
955
0.9691237941905083
956
0.9622664328330128
957
0.9546222536957486
958
0.9739834487377651
959
0.9558935989414561
960
0.918284427538997
961
0.9663888099910367
962
0.9669284540816931
963
0.9330682185788349
964
0.9909091367124017
965
0.9636098522225724
966
0.9296182155161881
967
0.9122653982997729
968
0.9157775983659353
969
0.9661309084944962
970
0.9596643182977881
971
0.9432646820061409
972
0.9501954993019022
973
0.8480048757756243
974
0.97291536971852
975
0.948778396018473
976
0.9422825279626476
977
0.9589548554484743
978
0.959759165613206
979
0.8090677611718391
980
0.9770776621876438
981
0.9654932330154663
982
0.93279663571

In [10]:
with open('/home/andrea/pythiaEvents/Event_0_qsolutions.dat', 'w') as f:
    for p in range(n_part):
        key = 's[%s]' %p
        f.write("%s\n" %(response.first.sample[key]))

In [17]:
p = 0
response.first.sample['s[7]']

0

In [ ]:
thrust = sum([ -1.0*coeff[i][j]*response.first.sample['s['+str(i)+']']*response.first.sample['s['+str(j)+']'] for i in range(0,n_part) for j in range(0,n_part)])
print(thrust)

In [24]:
sum_mom = sum([np.sqrt(px[i]*px[i] + py[i]*py[i]+ pz[i]*pz[i])  for i in range(0,n_part)])

In [25]:
print(sum_mom)

90.86611912138152


In [ ]:
-thrust/(2*sum_mom)

In [1]:
qubo

NameError: name 'qubo' is not defined